## Домашнее задание Кошкиной Ксении

# Домашнее задание 2. Извлечение коллокаций + NER

## Дедлайн
1 декабря 2021 (среда) 23:59 мск

## Описание

Выберите корпус отзывов на товары одной из категорий Amazon:
http://jmcauley.ucsd.edu/data/amazon/

(В низу страницы по ссылке есть код для загрузки данных, можете им воспользоваться)

Допустим, что вам нужно подготовить аналитический отчет по этим отзывам — например, для производителя нового продукта этой категории. Для этого будем искать упоминания товаров в отзывах (будем считать их NE). Учтите, что упоминание может выглядеть не только как "Iphone 10", но и как "модель", "телефон" и т.п.

**Важное замечание**: в задании приводятся примеры решений, вы можете их использовать!

1. (3 балла) Предложите 3 способа найти упоминания товаров в отзывах. 
Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара.
Могут помочь заголовки и дополнительные данные с Amazon (Metadata [здесь](https://nijianmo.github.io/amazon/index.html))
Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?

2. (2 балла) Реализуйте один из предложенных вами способов.

Примеры в качестве подсказки (можно использовать один из них): 
- написать правила с помощью [natasha/yargy](https://github.com/natasha/yargy)
- составить мини-словарь сущностей/дескрипторов, расширить с помощью эмбеддингов (например, word2vec)

3. (1 балл) Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

4. (3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). Не забудьте про частотный фильтр / сглаживание.
Выберите лучший результат (какая метрика  ранжирует выше коллокации, подходящие для отчёта).

5. (1 балл) Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров.
Должны получиться примерно такие группы:
```
watch 
--- 
stylish watch
good watches
great watch
love this watch
...
```

**Бонус** (2 балла): 
если придумаете способ объединить синонимичные упоминания (например, "Samsung Galaxy Watch", "watch", "smartwatch")

## Формат выполнения

jupyter-ноутбук + укажите, какие данные брали — или положите их тоже в репозиторий.

### 1. Три способа выделения коллокаций: 
1. обучить модель fasttext, использовать most_similar на частотных существительных, объединить их с топом существительных и с помощью spacy найти все коллокации. недостатки: модель должна обучаться на больших данных, а не как у меня...
2. можно сделать на основе прошлой домашки - использовать ключевые слова, но опыт показывает, что методы выделения ключевых слов не всегда справляются успешно.
3. использовать wordnet для выделения гипонимов нужных слов. недостатки: для редких слов гипонимов и синонимов почти нет.

### 2. Я выбрала корпус Digital Music.
Однако в процессе работы над корпусом обнаружила, что в основном в данном корпусе находятся отзывы к альбомам, а не к самому продукту (например, к пластинке или cd-диску). Это не повлияло на алгоритм работы, но коллокации содержали в себе слова, относящиеся, например, к исполнителю или песням или альбому. Возможно, следовало использовать метаданные.

In [1]:
import json
import gzip
import nltk
import spacy
from spacy.matcher import Matcher
from collections import Counter
from nltk.corpus import stopwords
from nltk.collocations import *
stopwords = stopwords.words('english')

In [2]:
reviews = []
with gzip.open('Digital_Music.json.gz', 'rt') as f_in:
    for el in f_in:
        reviews.append(json.loads(el.strip()))

In [3]:
all_reviews = reviews[:300000] #сократим, потому что отзывов слишком много

In [4]:
empty_reviews = []
for i in range(len(all_reviews)):
  try:
    element = all_reviews[i]['reviewText']
  except:
    empty_reviews.append(i)

In [5]:
clean_reviews = []
for i in range(len(all_reviews)):
  if i not in empty_reviews:
    clean_reviews.append(all_reviews[i]['reviewText'])

In [6]:
from gensim.models.fasttext import FastText

In [7]:
import re
def preprocess(text):
    text = text.lower().replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text)
    text = re.sub(' +', ' ', text)
    return text

In [8]:
reviews_model = []
for i in reviews[300000:1000000]:
    try:
        reviews_model.append(i['reviewText'])
    except:
        pass

In [9]:
reviews_for_model = []
for k in reviews_model:
    reviews_for_model.append(preprocess(k).split())

In [10]:
fasttext_model = FastText()
fasttext_model.build_vocab(reviews_for_model)
fasttext_model.train(reviews_for_model, total_examples=len(reviews_for_model), epochs=10)

(198595599, 271326570)

In [11]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
reviews_spacy = []
for el in clean_reviews:
    reviews_spacy.append(nlp(el.lower()))

In [12]:
lemmas = []
for el in reviews_spacy:
    for token in el:
        lemmas.append((token.lemma_, token.pos_))

In [13]:
common_words = Counter(lemmas).most_common(5000)

In [14]:
top_nouns = []
for i in common_words:
    if i[0][1] == 'NOUN':
        top_nouns.append(i[0][0])

In [15]:
top_noun = top_nouns[:10]

In [16]:
top_noun.remove('time')
top_noun.remove('voice')
top_noun.remove('year')
top_noun.remove('fan')

In [17]:
top_noun

['song', 'album', 'music', 'cd', 'track', 'band']

In [18]:
fasttext_model.wv.most_similar('nice')

[('eunice', 0.7834376096725464),
 ('great', 0.7746039032936096),
 ('cool', 0.769225001335144),
 ('good', 0.7671172022819519),
 ('pleasant', 0.7533867359161377),
 ('nicu', 0.7016269564628601),
 ('nicer', 0.6969878077507019),
 ('neat', 0.6916473507881165),
 ('terrific', 0.6893688440322876),
 ('funicello', 0.6849390268325806)]

In [19]:
model_words = []
for el in top_noun:
    model_words.append(el)
    for i in list(fasttext_model.wv.most_similar(el)[:4]):
        model_words.append(i[0])


In [43]:
adj = [{'POS': 'ADJ'},
       {"LEMMA": {"IN": model_words}}]
be = [{"LEMMA": {"IN": model_words}},
      {'LEMMA': 'be'},
      {'POS': {"IN": ['ADJ', 'NOUN']}}]
love = [{'LEMMA': 'love'},
        {'POS': {"IN": ['DET']}},
        {"LEMMA": {"IN": model_words}}]
matcher = Matcher(nlp.vocab)
matcher.add("adj", [adj])
matcher.add("be", [be])
matcher.add("love", [love])

In [49]:
matches = []
for doc in reviews_spacy:
    match = matcher(doc)
    for match_id, start, end in match:
        span = doc[start:end]
        matches.append(span.text)

In [50]:
common_collocations = Counter(matches).most_common()
common_collocations[:10]

[('love this song', 12657),
 ('great song', 9957),
 ('great music', 4163),
 ('great album', 3317),
 ('good song', 3316),
 ('favorite songs', 3176),
 ('beautiful song', 2617),
 ('good music', 2607),
 ('favorite song', 2350),
 ('great songs', 2247)]

In [51]:
_reviews = []
for i in common_collocations:
    for el in model_words:
        if el in i[0]:
            _reviews.append([el,i])

In [52]:
_reviews2 = []
for i in _reviews:
  if i not in _reviews2:
    _reviews2.append(i)

In [53]:
def get_all_colloc(word, reviews):
    word_list = []
    for el in reviews:
        if word in el[0]:
            word_list.append(el[1])
    return word_list

In [54]:
needed_words = ['album', 'song', 'music', 'cd']
for i in needed_words:
    for k in get_all_colloc(i, _reviews2)[:10]:
        print(*k)
    print('\n')

great album 3317
whole album 1803
first album 1650
love this album 1645
new album 1412
entire album 1347
best album 1175
good album 1063
best albums 767
other albums 744


love this song 12657
great song 9957
good song 3316
favorite songs 3176
beautiful song 2617
favorite song 2350
great songs 2247
best songs 2168
loved this song 2125
love the song 2081


great music 4163
good music 2607
classical music 970
christian music 792
beautiful music 753
love the music 711
new music 629
more music 528
music is great 442
real music 399


great cd 2126
love this cd 2099
whole cd 856
new cd 828
good cd 803
first cd 659
best cd 650
entire cd 635
cd is great 459
favorite cd 308




In [55]:
left = [{'IS_PUNCT': False},
        {"LEMMA": {"IN": model_words}}]
right = [{"LEMMA": {"IN": model_words}},
         {'IS_PUNCT': False}]
matcher2 = Matcher(nlp.vocab)
matcher2.add("left", [left])
matcher2.add("right", [right])
contexts = []
for doc in reviews_spacy:
    matches = matcher2(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        contexts.append(span.text)

In [56]:
Counter(contexts).most_common(10)

[('this song', 65282),
 ('this album', 33564),
 ('this cd', 24614),
 ('the song', 23133),
 ('the album', 17507),
 ('great song', 15264),
 ('the music', 15046),
 ('song is', 13858),
 ('the songs', 13719),
 ('the cd', 10878)]

Удаляем стоп-слова

In [57]:
ngrams = []
for el in contexts:
    stop = []
    for i in el.split():
        if i in stopwords:
            stop.append(i)
    if len(stop) == 0:
        ngrams.append(el.split())

In [58]:
ngrams[:5]

[['great', 'cd'],
 ['cd', 'full'],
 ['green', 'songs'],
 ['best', 'album'],
 ['favorite', 'songs']]

In [59]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder_bi = BigramCollocationFinder.from_documents(ngrams)

In [60]:
finder_bi.score_ngrams(bigram_measures.pmi)[:10]

[(('rx', 'bandits'), 18.42739264787276),
 (('tracker', 'e.p'), 18.42739264787276),
 (('cmr', 'albumes'), 16.842430147151603),
 (('owns', 'emusic'), 16.842430147151603),
 (('songza', 'stations'), 16.842430147151603),
 (('sv', 'albumns'), 16.842430147151603),
 (('cd3', 'alt'), 16.42739264787276),
 (('eyed', 'bandit'), 16.42739264787276),
 (('frequency', 'bandwidth'), 16.42739264787276),
 (('albume', 'drot'), 15.842430147151605)]

In [61]:
finder_bi.score_ngrams(bigram_measures.likelihood_ratio)[:10]

[(('song', 'song'), 32943.114344042886),
 (('great', 'song'), 23893.848277496345),
 (('song', 'music'), 20800.448225451888),
 (('music', 'song'), 20164.00643585248),
 (('album', 'song'), 15934.653813910345),
 (('song', 'album'), 15680.274056687758),
 (('songs', 'song'), 14044.054994619231),
 (('music', 'music'), 13066.046588394544),
 (('cd', 'song'), 10020.04388978883),
 (('album', 'music'), 10006.033201335726)]

In [62]:
finder_bi.score_ngrams(bigram_measures.raw_freq)[:10]

[(('great', 'song'), 0.02164912922674995),
 (('good', 'song'), 0.006941223692067234),
 (('great', 'music'), 0.005904436908474846),
 (('every', 'song'), 0.0055002177110414245),
 (("'s", 'music'), 0.004951330590316043),
 (('great', 'album'), 0.004705962445909089),
 (('favorite', 'songs'), 0.004504562003679104),
 (('great', 'cd'), 0.003745764562883033),
 (('beautiful', 'song'), 0.0037159799904405706),
 (('good', 'music'), 0.003697541921785713)]

In [63]:
finder_bi.score_ngrams(bigram_measures.chi_sq)[:10]


[(('rx', 'bandits'), 705063.0),
 (('tracker', 'e.p'), 352530.99999929086),
 (('cd3', 'alt'), 176264.24999574505),
 (('eyed', 'bandit'), 176264.24999574505),
 (('songza', 'stations'), 117509.66666548474),
 (('sv', 'albumns'), 117509.66666548474),
 (('cmr', 'albumes'), 117509.33333215141),
 (('owns', 'emusic'), 117509.33333215141),
 (('frequency', 'bandwidth'), 88131.99999875898),
 (("'ve", 'grouped'), 78337.77777604443)]